In [1]:
"""
Budyko–Sellers EBM with time-dependent, latitude-constant insolation.
Reads a 1D insolation file, evolves surface temperature, computes cost,
and generates plots.
"""

import numpy as np
import matplotlib.pyplot as plt

def budyko_sellers_time_dep(XXS, insol, DT, N):
    """
    Run Budyko–Sellers EBM with time-varying insolation.

    Parameters
    ----------
    XXS : array_like, shape (2*N,)
        Control vector [Δα_logit (N), Δε_logit (N)]
    insol : array_like, shape (NT,)
        Insolation time series (constant in latitude).
    DT : float
        Time step (seconds).
    N : int
        Number of latitude bands.
    Returns
    -------
    T : ndarray, shape (N,)
        Final temperature profile (K).
    """
    # Grid
    Xedges = np.linspace(-1, 1, N+1)
    X = 0.5 * (Xedges[:-1] + Xedges[1:])
    DX = X[1] - X[0]
    LAT = np.degrees(np.arcsin(X))

    # Constants
    SIGMA = 5.67e-8
    EPSILON = 0.63
    DIFF = 0.6

    # Split controls
    dalpha = XXS[:N]
    demiss = XXS[N:]

    # Initial T
    T = 288.0 + 60.0*(1 - X**2) - 20.0

    # Time integration
    for val in insol:
        SX = val
        # Albedo
        alpha0 = 0.354 + 0.25*(1.5*np.sin(np.radians(LAT))**2 - 0.5)
        alpha_logit = np.log(alpha0/(1-alpha0)) + dalpha
        ALPHA = 1.0/(1.0 + np.exp(-alpha_logit))
        # Emissivity
        emiss_logit = np.log(EPSILON/(1-EPSILON)) + demiss
        emiss = 1.0/(1.0 + np.exp(-emiss_logit))

        FIN = SX * (1 - ALPHA)
        FOUT = emiss * SIGMA * T**4

        # Diffusion
        FDIFF = np.zeros_like(T)
        for i in range(N):
            if i == 0:
                FDIFF[i] = DIFF*(1 - Xedges[1]**2)*(T[i+1]-T[i])/(DX*DX)
            elif i == N-1:
                FDIFF[i] = -DIFF*(1 - Xedges[N]**2)*(T[i]-T[i-1])/(DX*DX)
            else:
                FDIFF[i] = DIFF*((1 - Xedges[i+1]**2)*(T[i+1]-T[i]) - (1 - Xedges[i]**2)*(T[i]-T[i-1]))/(DX*DX)

        T = T + DT*(FIN - FOUT + FDIFF)

    return T


In [13]:

# Load data
insolation = np.loadtxt('insolation.dat')  # (NT,)
TARGET_DT = np.loadtxt('T_zonalmean_NCEP.dat')  # (N,)
N = TARGET_DT.size
DT = 1

# Initial control
XXS0 = np.zeros(2*N)

# Run model
T_final = budyko_sellers_time_dep(XXS0, insolation[0:1], DT, N)

# Compute cost (K -> °C)
J = np.sqrt(np.sum((T_final - TARGET_DT - 273.0)**2))
print(f"Final cost J = {J:.4f}")

# Plot final temperature profile
Xedges = np.linspace(-1, 1, N+1)
X = 0.5 * (Xedges[:-1] + Xedges[1:])
LAT = np.degrees(np.arcsin(X))


Final cost J = 1217.9128


In [15]:
plt.plot(T_final-273)

array([  89.23367314,   81.61039574,   73.9731792 ,   66.32973871,
         58.68789558,   51.05556357,   43.44073521,   35.85146804,
         28.29587092,   20.78209027,   13.31829642,    5.91266992,
         -1.42661197,   -8.69138874,  -15.87353001,  -22.96494876,
        -29.95761436,  -36.84356551,  -43.61492297,  -50.26390206,
        -56.78282501,  -63.164133  ,  -69.40039807,  -75.48433461,
        -81.40881076,  -87.16685937,  -92.75168876,  -98.15669313,
       -103.37546264, -108.40179316, -113.22969574, -117.85340556,
       -122.26739075, -126.46636058, -130.44527349, -134.19934457,
       -137.7240527 , -141.01514729, -144.06865462, -146.88088365,
       -149.44843155, -151.7681887 , -153.83734327, -155.65338536,
       -157.21411068, -158.51762383, -159.56234103, -160.34699246,
       -160.87062414, -161.13259928, -161.13259928, -160.87062414,
       -160.34699246, -159.56234103, -158.51762383, -157.21411068,
       -155.65338536, -153.83734327, -151.7681887 , -149.44843